In [ ]:
import PyPDF2
import pandas as pd
import os
import glob

In [ ]:
path = '../INVOICES/202011/'

facturas = glob.glob(path + '*.pdf')
Countries = ['COSTA RICA', 'Dominican Repub', 'Ecuador', 'Guatemala', 'Jamaica', 'Mexico', 'Panama', 'Peru', 'Puerto Rico'
            'Paraguay', 'El Salvador', 'Trinidad', 'Uruguay', 'COLOMBIA', 'Honduras']

os.mkdir(path + 'facturas_csv/')

In [ ]:
%%time
for factura in facturas:
    pdfFileObj = open(factura, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    number_of_pages = pdfReader.getNumPages()
    text = ''
    
    for i in range(number_of_pages):
        
        pageObj = pdfReader.getPage(i)
        text = text + pageObj.extractText()
        
    for country in Countries:
        if country in text:
            fact = text[text.find(': ') + 2:text.find(' y el')].replace('/', '-') + '_' + country.capitalize()
            print(fact)
            break
            
    Conceptos = []
    Valor = []
    
    for concepto in ['TTS Services', 'ICG Regional', 'GSG Client Desk', 'Data Managemet and Analytics', 'TWCF', 'LIH']:
        
        pos_concepto = text.find(concepto)
        if pos_concepto == -1:
            continue
            
        if 'Colombia' in fact:
            moneda = 'COP'
        else:
            moneda = 'USD'
            
        pos_valor = text[pos_concepto:].find(moneda)+3+pos_concepto
        
        concepto_text = text[pos_valor:]
        
        valor = concepto_text[:concepto_text.find('.')+3]
        
        iva = text.find('IVA')
        
        iva = text[iva+3 :iva+7]
        
        Conceptos.append(concepto)
        Valor.append(valor.replace(',', ''))
        
        Conceptos.append('IVA')
        IVA = float(iva)*float(str(valor).replace(',', ''))/100
        Valor.append(round(IVA,2))
        
        Conceptos.append('Mark up 10%')
        markup = float(str(valor).replace(',', ''))*0.1
        Valor.append(round(markup, 2))
        
        Conceptos.append('IVA')
        IVA = float(iva)*markup/100
        Valor.append(round(IVA,2))
        
        text = text[text.find(concepto)+1:]
        
    df_1 = pd.DataFrame({'Concepto': Conceptos, 'Valor':Valor})
    
    Conceptos = []
    Valor = []
    
    for concepto in ['SUBTOTAL', 'IMPUESTOS', 'TOTAL']:
        pos_concepto = text.find(concepto)
        pos_valor = text[pos_concepto:].find('.')+3+pos_concepto
        concepto_text = text[pos_concepto:pos_valor]
        valor = concepto_text[len(concepto):]
        Conceptos.append(concepto)
        Valor.append(valor.replace(',', ''))
        text = text[text.find(concepto):]
        
    df_2 = pd.DataFrame({'Concepto': Conceptos, 'Valor': Valor})    
    df = pd.concat([df_1, df_2])
    df.to_csv(path + 'facturas_csv/' + fact + '.csv', index = False, sep = ';')